In [ ]:
import subprocess
command = ["bin/ffmpeg", "-f", "concat", "-i", "bin/files.txt", "-vcodec", "copy", "-acodec", "copy", "test.mp4"]

try:
    result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Wystąpił błąd podczas tworzenia pliku final_file.mp4: {str(e)}")



In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import os
import subprocess


class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.filename = "final_file.mp4"
        self.master = master
        self.pack()
        self.create_widgets()

    def marge_videos(self): 
        print(self.filename)
        command = ["bin/ffmpeg", "-f", "concat", "-i", "bin/files.txt", "-vcodec", "copy", "-acodec", "copy", self.filename]

        try:
            result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            print(result.stdout.decode())
        except subprocess.CalledProcessError as e:
            print(f"Error: {str(e)}")

    def create_widgets(self):
        self.select_button = tk.Button(self)
        self.select_button["text"] = "Add MP4 files"
        self.select_button["command"] = self.select_files
        self.select_button.pack(side="top", fill="both", expand=True)

        self.file_listbox = tk.Listbox(self)
        self.file_listbox.pack(side="top", fill="both", expand=True)

        self.output_label = tk.Label(self, text="Destination path:")
        self.output_label.pack(side="top")
        self.output_entry = tk.Entry(self)
        self.output_entry.pack(side="top", fill="both", expand=True)

        self.output_button = tk.Button(self)
        self.output_button["text"] = "Add destination file"
        self.output_button["command"] = self.select_output
        self.output_button.pack(side="top", fill="both", expand=True)

        self.save_button = tk.Button(self)
        self.save_button["text"] = "Start processing"
        self.save_button["command"] = self.save_to_file
        self.save_button.pack(side="top", fill="both", expand=True)

    def select_files(self):
        filenames = filedialog.askopenfilenames(filetypes=[('MP4 files', '*.mp4')])
        filenames = sorted(filenames, key=os.path.getctime)
        self.file_listbox.delete(0, tk.END)
        for filename in filenames:
            self.file_listbox.insert(tk.END, os.path.basename(filename))

    def select_output(self):
        self.filename = filedialog.asksaveasfilename(defaultextension=".mp4")
        self.output_entry.delete(0, tk.END)
        self.output_entry.insert(0, self.filename)

    def save_to_file(self):
        output_file = self.output_entry.get()
        if not output_file:
            messagebox.showerror("Error", "Please enter the path to the resulting file.")
            return
        filenames = self.file_listbox.get(0, tk.END)
        with open('./bin/files.txt', 'w') as f:
            for filename in filenames:
                f.write(f"file {os.path.relpath(filename)}\n")
        self.marge_videos()
        messagebox.showinfo("Success", "The files have been merged.")

root = tk.Tk()
root.wm_title("QMV Maker v1.0")
root.wm_iconbitmap('bin/icomax.ico')
root.geometry("500x500")  # Ustawienie rozmiaru okna na 500x500 pikseli
app = Application(master=root)
app.mainloop()

In [ ]:
import os
import glob
import argparse
import subprocess
from pathlib import Path

def get_mp4_files(directory):
    """Pobiera listę plików MP4 z katalogu i sortuje je według daty utworzenia."""
    mp4_files = glob.glob(os.path.join(directory, "*.MP4"))
    # Sortuj pliki według daty utworzenia (od najstarszych do najnowszych)
    mp4_files.sort(key=lambda x: os.path.getctime(x))
    return mp4_files

def create_ffmpeg_input_file(mp4_files, output_txt="files.txt"):
    """Tworzy plik tekstowy z listą ścieżek do plików MP4 w formacie FFmpeg."""
    with open(output_txt, "w", encoding="utf-8") as f:
        for mp4_file in mp4_files:
            # Konwertuj ścieżkę na format z ukośnikami i dodaj cudzysłowy
            mp4_path = mp4_file.replace("//", "/")
            f.write(f"file '{mp4_path}'/n")

def run_ffmpeg(ffmpeg_path, input_txt="files.txt", output_file="final_file.mp4"):
    """Uruchamia FFmpeg, aby połączyć pliki MP4 w jeden plik."""
    command = [
        ffmpeg_path,
        "-f", "concat",
        "-safe", "0",
        "-i", input_txt,
        "-vcodec", "copy",
        "-acodec", "copy",
        output_file
    ]
    try:
        subprocess.run(command, check=True, text=True, capture_output=True)
        print(f"Pomyślnie utworzono plik: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Błąd podczas uruchamiania FFmpeg: {e.stderr}")

def main():
    # Parsowanie argumentów wiersza poleceń
    parser = argparse.ArgumentParser(description="Łączenie plików MP4 z GoPro za pomocą FFmpeg.")
    parser.add_argument("directory", help="Ścieżka do katalogu z plikami MP4")
    parser.add_argument("--ffmpeg", default="ffmpeg", help="Ścieżka do pliku wykonywalnego FFmpeg")
    args = parser.parse_args()

    # Sprawdź, czy katalog istnieje
    if not os.path.isdir(args.directory):
        print(f"Błąd: Katalog {args.directory} nie istnieje!")
        return

    # Pobierz i posortuj pliki MP4
    mp4_files = get_mp4_files(args.directory)
    if not mp4_files:
        print(f"Nie znaleziono plików MP4 w katalogu: {args.directory}")
        return

    print(f"Znalezione pliki MP4 ({len(mp4_files)}):")
    for mp4 in mp4_files:
        print(f" - {mp4}")

    # Utwórz plik files.txt
    create_ffmpeg_input_file(mp4_files)
    print(f"Utworzono plik: files.txt")

    # Uruchom FFmpeg
    run_ffmpeg(args.ffmpeg)

if __name__ == "__main__":
    main()

### Dzialajacy skrypt - przycina i łączy z gorpo

In [2]:
import os
import glob
import subprocess

# Define parameters
directory = "G:/DCIM/100GOPRO"  # Ścieżka do katalogu z plikami MP4
ffmpeg_path = "bin/ffmpeg.exe"  # Ścieżka do ffmpeg.exe
top = 200  # Margines górny w pikselach
bottom = 1100  # Margines dolny w pikselach
left = 0  # Margines lewy w pikselach
right = 0  # Margines prawy w pikselach
output_txt = "bin/files.txt"  # Nazwa pliku tekstowego z listą plików
output_file = "final_file.mp4"  # Nazwa pliku wyjściowego

def get_mp4_files(directory):
    """Pobiera listę plików MP4 z katalogu i sortuje je według daty utworzenia."""
    mp4_files = glob.glob(os.path.join(directory, "*.MP4"))
    mp4_files.sort(key=lambda x: os.path.getctime(x))
    return mp4_files

def create_ffmpeg_input_file(mp4_files, output_txt):
    """Tworzy plik tekstowy z listą ścieżek do plików MP4 w formacie FFmpeg."""
    with open(output_txt, "w", encoding="utf-8") as f:
        for mp4_file in mp4_files:
            mp4_path = mp4_file.replace("\\", "/")
            f.write(f"file '{mp4_path}'\n")

def get_video_dimensions(ffmpeg_path, video_file):
    """Pobiera wymiary wideo za pomocą ffprobe."""
    if os.path.isabs(ffmpeg_path):
        ffprobe_path = os.path.join(os.path.dirname(ffmpeg_path), "ffprobe.exe")
    else:
        ffprobe_path = "ffprobe"
    command = [ffprobe_path, "-v", "error", "-select_streams", "v:0", "-show_entries", "stream=width,height", "-of", "csv=p=0", video_file]
    try:
        output = subprocess.check_output(command, text=True).strip()
        width, height = map(int, output.split(","))
        return width, height
    except Exception as e:
        print(f"Błąd podczas pobierania wymiarów wideo: {e}")
        raise

def run_ffmpeg(ffmpeg_path, input_txt, output_file, crop_filter=None):
    """Uruchamia FFmpeg, aby połączyć pliki MP4 w jeden plik, opcjonalnie z przycinaniem."""
    command = [
        ffmpeg_path,
        "-f", "concat",
        "-safe", "0",
        "-i", input_txt,
    ]
    if crop_filter:
        command += ["-vf", crop_filter, "-c:v", "libx264"]
    else:
        command += ["-c:v", "copy"]
    command += ["-c:a", "copy", output_file]
    try:
        subprocess.run(command, check=True)
        print(f"Pomyślnie utworzono plik: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Błąd podczas uruchamiania FFmpeg: {e}")

# Main script
mp4_files = get_mp4_files(directory)
if not mp4_files:
    print(f"Nie znaleziono plików MP4 w katalogu: {directory}")
else:
    print(f"Znalezione pliki MP4 ({len(mp4_files)}):")
    for mp4 in mp4_files:
        print(f" - {mp4}")
    create_ffmpeg_input_file(mp4_files, output_txt)
    print(f"Utworzono plik: {output_txt}")
    if top > 0 or bottom > 0 or left > 0 or right > 0:
        first_video = mp4_files[0]
        width, height = get_video_dimensions(ffmpeg_path, first_video)
        out_w = width - left - right
        out_h = height - top - bottom
        if out_w <= 0 or out_h <= 0:
            print("Error: Crop margins are too large, resulting in non-positive dimensions.")
        else:
            crop_filter = f"crop={out_w}:{out_h}:{left}:{top}"
            run_ffmpeg(ffmpeg_path, output_txt, output_file, crop_filter)
    else:
        run_ffmpeg(ffmpeg_path, output_txt, output_file)

Znalezione pliki MP4 (6):
 - G:/DCIM/100GOPRO\GX020211.MP4
 - G:/DCIM/100GOPRO\GX030211.MP4
 - G:/DCIM/100GOPRO\GX040211.MP4
 - G:/DCIM/100GOPRO\GX010212.MP4
 - G:/DCIM/100GOPRO\GX010213.MP4
 - G:/DCIM/100GOPRO\GX020213.MP4
Utworzono plik: bin/files.txt


Pomyślnie utworzono plik: final_file.mp4


### próba korekty obiektywu fisheye 

In [4]:
import os
import glob
import subprocess

# Define parameters
#directory = "G:/DCIM"  # Ścieżka do katalogu z plikami MP4
directory ="F:/test"
ffmpeg_path = "bin/ffmpeg.exe"  # Ścieżka do ffmpeg.exe
k1 = 0  # Współczynnik zniekształceń radialnych (sugerowane: -0.2 do -0.4)
k2 = 0.5   # Drugi współczynnik zniekształceń (sugerowane: 0.0 do 0.1)
top = 200  # Margines górny w pikselach
bottom = 1100  # Margines dolny w pikselach
left = 0  # Margines lewy w pikselach
right = 0  # Margines prawy w pikselach
output_txt = "bin/files.txt"  # Nazwa pliku tekstowego z listą plików
output_file = "final_file.mp4"  # Nazwa pliku wyjściowego

def get_mp4_files(directory):
    """Pobiera listę plików MP4 z katalogu i sortuje je według daty utworzenia."""
    mp4_files = glob.glob(os.path.join(directory, "*.MP4"))
    mp4_files.sort(key=lambda x: os.path.getctime(x))
    return mp4_files

def create_ffmpeg_input_file(mp4_files, output_txt):
    """Tworzy plik tekstowy z listą ścieżek do plików MP4 w formacie FFmpeg."""
    with open(output_txt, "w", encoding="utf-8") as f:
        for mp4_file in mp4_files:
            mp4_path = mp4_file.replace("\\", "/")
            f.write(f"file '{mp4_path}'\n")

def get_video_dimensions(ffmpeg_path, video_file):
    """Pobiera wymiary wideo za pomocą ffprobe."""
    if os.path.isabs(ffmpeg_path):
        ffprobe_path = os.path.join(os.path.dirname(ffmpeg_path), "ffprobe.exe")
    else:
        ffprobe_path = "ffprobe"
    command = [ffprobe_path, "-v", "error", "-select_streams", "v:0", "-show_entries", "stream=width,height", "-of", "csv=p=0", video_file]
    try:
        output = subprocess.check_output(command, text=True).strip()
        width, height = map(int, output.split(","))
        return width, height
    except Exception as e:
        print(f"Błąd podczas pobierania wymiarów wideo: {e}")
        raise

def run_ffmpeg(ffmpeg_path, input_txt, output_file, filters=None):
    """Uruchamia FFmpeg, aby połączyć pliki MP4 w jeden plik, opcjonalnie z filtrami."""
    command = [
        ffmpeg_path,
        "-f", "concat",
        "-safe", "0",
        "-i", input_txt,
    ]
    if filters:
        command += ["-vf", ",".join(filters), "-c:v", "libx264"]
    else:
        command += ["-c:v", "copy"]
    command += ["-c:a", "copy", output_file]
    try:
        subprocess.run(command, check=True)
        print(f"Pomyślnie utworzono plik: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Błąd podczas uruchamiania FFmpeg: {e}")

# Main script
mp4_files = get_mp4_files(directory)
if not mp4_files:
    print(f"Nie znaleziono plików MP4 w katalogu: {directory}")
else:
    print(f"Znalezione pliki MP4 ({len(mp4_files)}):")
    for mp4 in mp4_files:
        print(f" - {mp4}")
    create_ffmpeg_input_file(mp4_files, output_txt)
    print(f"Utworzono plik: {output_txt}")
    
    filters = []
    if k1 != 0 or k2 != 0:
        filters.append(f"lenscorrection=k1={k1}:k2={k2}")
    
    if top > 0 or bottom > 0 or left > 0 or right > 0:
        first_video = mp4_files[0]
        width, height = get_video_dimensions(ffmpeg_path, first_video)
        out_w = width - left - right
        out_h = height - top - bottom
        if out_w <= 0 or out_h <= 0:
            print("Error: Crop margins are too large, resulting in non-positive dimensions.")
        else:
            filters.append(f"crop={out_w}:{out_h}:{left}:{top}")
    
    if filters:
        run_ffmpeg(ffmpeg_path, output_txt, output_file, filters)
    else:
        run_ffmpeg(ffmpeg_path, output_txt, output_file)

Znalezione pliki MP4 (1):
 - F:/test\GX010211 (1)-trimmed.MP4
Utworzono plik: bin/files.txt
Pomyślnie utworzono plik: final_file.mp4


### WINDOWS & UBUNTU

In [ ]:
import os
import glob
import subprocess
import platform

# Define parameters
directory = "/media/mk/011F-B391/DCIM/100GOPRO/" if platform.system() == "Linux" else "G:/DCIM/100GOPRO"
top = 0  # Margines górny w pikselach
bottom = 700  # Margines dolny w pikselach
left = 0  # Margines lewy w pikselach
right = 0  # Margines prawy w pikselach
output_txt = "files.txt"  # Nazwa pliku tekstowego z listą plików
output_file = "final_file.mp4"  # Nazwa pliku wyjściowego

# Wykrywanie systemu operacyjnego
system = platform.system()
if system == "Windows":
    ffmpeg_path = "bin/ffmpeg.exe"
    ffprobe_path = "bin/ffprobe.exe"
elif system == "Linux":
    ffmpeg_path = "ffmpeg"
    ffprobe_path = "ffprobe"
else:
    raise OSError("Nieobsługiwany system operacyjny")

# Sprawdzenie istnienia plików wykonywalnych pod Windows
if system == "Windows":
    if not os.path.isfile(ffmpeg_path):
        raise FileNotFoundError(f"FFmpeg nie znaleziono w {ffmpeg_path}")
    if not os.path.isfile(ffprobe_path):
        raise FileNotFoundError(f"ffprobe nie znaleziono w {ffprobe_path}")

def get_mp4_files(directory):
    """Pobiera listę plików MP4 z katalogu i sortuje je według daty utworzenia."""
    mp4_files = glob.glob(os.path.join(directory, "*.MP4"))
    mp4_files.sort(key=lambda x: os.path.getctime(x))
    return mp4_files

def create_ffmpeg_input_file(mp4_files, output_txt):
    """Tworzy plik tekstowy z listą ścieżek do plików MP4 w formacie FFmpeg."""
    with open(output_txt, "w", encoding="utf-8") as f:
        for mp4_file in mp4_files:
            mp4_path = mp4_file.replace("\\", "/")  # Zamiana backslashe na ukośniki
            f.write(f"file '{mp4_path}'\n")

def get_video_dimensions(ffprobe_path, video_file):
    """Pobiera wymiary wideo za pomocą ffprobe."""
    command = [ffprobe_path, "-v", "error", "-select_streams", "v:0", "-show_entries", "stream=width,height", "-of", "csv=p=0", video_file]
    try:
        output = subprocess.check_output(command, text=True).strip()
        width, height = map(int, output.split(","))
        return width, height
    except Exception as e:
        print(f"Błąd podczas pobierania wymiarów wideo: {e}")
        raise

def run_ffmpeg(ffmpeg_path, input_txt, output_file, crop_filter=None):
    """Uruchamia FFmpeg, aby połączyć pliki MP4 w jeden plik, opcjonalnie z przycinaniem."""
    command = [
        ffmpeg_path,
        "-f", "concat",
        "-safe", "0",
        "-i", input_txt,
    ]
    if crop_filter:
        command += ["-vf", crop_filter, "-c:v", "libx264"]
    else:
        command += ["-c:v", "copy"]
    command += ["-c:a", "copy", output_file]
    try:
        subprocess.run(command, check=True)
        print(f"Pomyślnie utworzono plik: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Błąd podczas uruchamiania FFmpeg: {e}")

# Main script
mp4_files = get_mp4_files(directory)
if not mp4_files:
    print(f"Nie znaleziono plików MP4 w katalogu: {directory}")
else:
    print(f"Znalezione pliki MP4 ({len(mp4_files)}):")
    for mp4 in mp4_files:
        print(f" - {mp4}")
    create_ffmpeg_input_file(mp4_files, output_txt)
    print(f"Utworzono plik: {output_txt}")
    if top > 0 or bottom > 0 or left > 0 or right > 0:
        first_video = mp4_files[0]
        width, height = get_video_dimensions(ffprobe_path, first_video)
        out_w = width - left - right
        out_h = height - top - bottom
        if out_w <= 0 or out_h <= 0:
            print("Błąd: Marginesy przycinania są za duże, co prowadzi do niepoprawnych wymiarów.")
        else:
            crop_filter = f"crop={out_w}:{out_h}:{left}:{top}"
            run_ffmpeg(ffmpeg_path, output_txt, output_file, crop_filter)
    else:
        run_ffmpeg(ffmpeg_path, output_txt, output_file)